# Global Solution 2 - RPA

Integrantes:<br>
- Henrique Marra Barbosa - RM97672
- Arthur Hieda Cunha - RM551882
- Lucas Bueno Taets Gustavo - RM552162

### Imports

In [23]:
import requests
import sqlite3
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

### Configurações

In [24]:
# Configurações de APIs
ENERGY_API_URL = "https://api.eia.gov/v2/electricity/retail-sales/data/"
EIA_API_KEY = "gEdTQHlS1PksCEtK9LC6mz1BwO7luKW5YdJ1Yppg"
OPENMETEO_API_URL = "https://archive-api.open-meteo.com/v1/era5"
LATITUDE = 36.7783  # Latitude da Califórnia
LONGITUDE = -119.4179  # Longitude da Califórnia

# Banco de Dados SQLite
DB_NAME = "energia_sustentavel.db"

### Coleta de Dados

In [25]:
# Função para converter data para timestamp UNIX
def convert_date_to_timestamp(date_str):
    date_time_obj = time.strptime(date_str, "%Y-%m-%d")
    return int(time.mktime(date_time_obj))

# Função para buscar dados de energia
def fetch_energy_data():
    params = {
        "api_key": EIA_API_KEY,
        "frequency": "monthly",
        "data[0]": "price",
        "facets[stateid][]": "CA",
        "start": "2020-01",  # Alterado para iniciar em 2020
        "end": "2023-12",    # Alterado para terminar em 2024
        "sort[0][column]": "period",
        "sort[0][direction]": "asc"
    }
    response = requests.get(ENERGY_API_URL, params=params)
    if response.status_code == 200:
        return response.json()['response']['data']
    else:
        raise Exception(f"Erro ao acessar a API da EIA: {response.status_code}")

# Função para buscar dados do clima usando Open-Meteo API
def fetch_weather_data():
    params = {
        "latitude": LATITUDE,
        "longitude": LONGITUDE,
        "start_date": "2020-01-01",  # Alterado para iniciar em 2020
        "end_date": "2023-12-31",    # Alterado para terminar em 2024
        "hourly": "temperature_2m"   # Dados de temperatura a cada hora
    }
    
    response = requests.get(OPENMETEO_API_URL, params=params)
    if response.status_code == 200:
        weather_data = response.json()
        time_data = weather_data['hourly']['time']
        temp_data = weather_data['hourly']['temperature_2m']
        
        # Formatar os dados em uma lista de dicionários
        weather_list = [{"date": time_data[i], "temp_c": temp_data[i]} for i in range(len(time_data))]
        return weather_list
    else:
        raise Exception(f"Erro ao acessar a API do Open-Meteo: {response.status_code}")

### Tratamento de Dados

In [26]:
# Função para processar dados
def process_data(energy_data, weather_data):
    # Dados de energia
    energy_df = pd.DataFrame(energy_data)
    energy_df['period'] = pd.to_datetime(energy_df['period'])
    
    # Convertendo a coluna 'price' para tipo numérico (float)
    energy_df['price'] = pd.to_numeric(energy_df['price'], errors='coerce')
    energy_df = energy_df[['period', 'price']].sort_values('period')
    
    # Dados de clima
    weather_df = pd.DataFrame(weather_data)
    weather_df['date'] = pd.to_datetime(weather_df['date'])
    
    # Convertendo a coluna 'temp_c' para tipo numérico (float)
    weather_df['temp_c'] = pd.to_numeric(weather_df['temp_c'], errors='coerce')
    weather_df = weather_df.sort_values('date')
    
    # Padronização e união dos dados
    # Usando a função merge_asof para garantir o alinhamento temporal
    merged_df = pd.merge_asof(energy_df, weather_df, left_on='period', right_on='date')
    
    # Verifique se há alguma falta de dados após a mesclagem
    merged_df = merged_df.dropna()  # Remove linhas com dados faltantes

    return merged_df

### Armazenamento e Modelagem

In [27]:
# Função para salvar dados
def save_to_storage(dataframe):
    # Exportar para CSV
    dataframe.to_csv("energia_sustentavel.csv", index=False)
    
    # Salvar no SQLite
    conn = sqlite3.connect(DB_NAME)
    dataframe.to_sql("consumo_energia", conn, if_exists="replace", index=False)
    conn.close()

# Função para treinar modelo
def train_model(dataframe):
    # Selecione as variáveis independentes (temperatura)
    X = dataframe[['temp_c']]
    
    # Selecione a variável dependente (preço da energia)
    y = dataframe['price']
    
    # Divisão dos dados em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Inicialize o modelo de regressão linear
    model = LinearRegression()
    
    # Treinamento do modelo
    model.fit(X_train, y_train)
    
    # Avaliação do modelo
    score = model.score(X_test, y_test)
    print(f"Precisão do modelo: {score:.2f}")
    
    return model

# Função para plotar previsões
def plot_predictions(model, dataframe):
    plt.figure(figsize=(10, 6))
    plt.scatter(dataframe['temp_c'], dataframe['price'], color='blue', label='Dados reais')
    plt.plot(dataframe['temp_c'], model.predict(dataframe[['temp_c']]), color='red', label='Previsão')
    plt.xlabel('Temperatura (°C)')
    plt.ylabel('Preço da Energia')
    plt.title('Previsão de Preço da Energia baseada na Temperatura')
    plt.legend()
    plt.savefig('previsao_energia.png')
    plt.close()

### Execução Principal

In [28]:
if __name__ == "__main__":
    try:
        # Coletar dados das fontes
        energy_data = fetch_energy_data()
        weather_data = fetch_weather_data()

        # Processar os dados
        processed_data = process_data(energy_data, weather_data)

        # Armazenar os dados
        save_to_storage(processed_data)

        # Treinar modelo e fazer previsões
        model = train_model(processed_data)
        plot_predictions(model, processed_data)

        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Erro: {e}")

Precisão do modelo: -0.15
Processo concluído com sucesso!
